<a href="https://colab.research.google.com/github/AlanAmaro13/Transistor_Characterization/blob/main/Transistor_Characterization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caracterización de transistores

Hola! Este código está hecho para realizar la caracterízación de parámetros de un amplificador con transistor de emisor o base común a partir de los valores de:


*   VCC - Voltaje de la fuente directa
*   RE - Resistencia del emisor
*   R1 - Resistencia 1 del divisor de voltaje
*   R2 - Resistencia 2 del divisor de voltaje
*   RC - Resistencia del colector
*   BETA - Valor beta del transistor
*   RL - Valor de la resistencia de carga
*   CC - Valor del capacitor del colector
*   CE - Valor del capacitor del emisor
*   CB - Valor del capacitor de la base

A partir de las siguientes expresiones:

\begin{equation}
\begin{split}
    V_{BB} &= \frac{V_{CC} R_{2} }{R_{1} + R_{2}} \\\\
    R_{B} &= \frac{R_{2}R_{1}}{R_{2} + R_{1}} \\\\
    V_{BE} &= 0.7 V \\ \\
    i_{B} &= \frac{V_{BB} - V_{BE}}{ R_{B} + R_{E}(\beta + 1) } \\\\
    i_{c} &= \beta i_{B} \\ \\
    V_{CE} &= V_{CC} - [ R_{c} \beta + R_{E}(\beta + 1) ]i_{B} \\\\
    h_{fe} &= \beta \\ \\
    h_{ie} &= \frac{0.026}{i_{B}} \\
    G_{V} &= \frac{h_{fe}}{h_{ie}}(R_{C}||R_{L}) \\ \\
    G_{I} &= h_{fe} (\frac{R_{B}}{R_{B} + h_{ie}})(\frac{R_{C}}{R_{C} + R_{L}}) \\ \\
    z_{i} &= R_{B}||h_{ie} \\ \\
    z_{o} &= R_{C} \\\\
    f_{CB}& = \frac{1}{2\pi R_{B}C_{B}} \\ \\
    f_{CE} &= \frac{1}{2\pi R_{E}C_{E}} \\\\
    f_{CC} &= \frac{1}{2\pi R_{C}C_{C}} \\\\
    |V_{i - Max}| &= \frac{V_{CC}}{G_{V}}
\end{split}
\end{equation}
Para los transistores con base común, el análisis de DC es análogo, y las expresiones para AC corresponden a:
\begin{equation}
\begin{split}
    h_{fb} &=\alpha = \frac{\beta}{\beta + 1} \\ \\
    h_{ib} &= \frac{0.026 }{(\beta+1)i_{B}} \\ \\
    G_{V} &= \frac{ h_{fb} }{ h_{ib} } (R_{c}||R_{L}) \\ \\
    G_{I} &= h_{fb}(\frac{R_{C}}{R_{C} + R_{L} })(\frac{R_{E}}{R_{E} + h_{ib}}) \\ \\
    z_{i} &= R_{E}||h_{ib} \\ \\
    z_{o} &= R_{C}
\end{split}
\end{equation}







In [ ]:
! pip install prettytable

In [ ]:
# Consideramos los siguientes datos como dados
from math import pi
from prettytable import PrettyTable
table = PrettyTable()

# Configuración del transistor
config = str(input("¿En qué configuración está el amplificador\n Emisor o base común? [e, b]: "))
r  = int(input("¿Cuál es la cantidad de resistencias en la divisora? [1,2,3]: "))

r1 = 1.0e6
r2 = 8.2e3
r3 = 10.0e3

# Valor base
#vcc = 15.0
#beta = 75.0
vcc = 9
beta = 200

# If you're using a resistor divisor:
#r1 = 100.0e3
#r2 = 15.0e3
#rb = 1e6

# Valores de las resistencias del transistor
#re = 680.0
#rc = 4.7e3
re = 560.0
rc = 3.9e3

# Resistencia de carga
rl = 6.8e3

cb = 200.0e-6
ce = 10.0e-6
cc = 1.0e-6


def rb_vbb_calculator(r):
  if r == 1:
    rb = r1
    vbb = vcc

  elif r==2:
    rb = (r1*r2) / (r1+r2)
    vbb =  (vcc*r2)/(r1 + r2)

  elif r==3:
    rb = ((r1 *r2) / (r1 + r2)) + r3
    vbb =  vcc * (r3 / (rb+r3))

  return [rb, vbb]

rb = rb_vbb_calculator(r)
rb = rb[0]


value_initial = [vcc, beta, r1, r2, r3, rb, re, rc, rl,
                 cb, ce, cc]

names = ["VCC", "BETA", "R1", "R2", "R3", "RB", "RE", "RC", "RL",
         "CB", "CE", "CC", "VBB", "VBE", "IB", "IC", "VCE",
         "HFE", "HIE", "GV", "GI", "ZI", "ZO", "FCB", "FCE",
         "FCC", "F_MIN", "VI-MAX"]



def amplificador_emisor_comun(vcc, beta, rb, re, rc, rl, cb, ce, cc):
  vbe = 0.7

  vbb = rb_vbb_calculator(r)
  vbb = vbb[1]

  ib = (vbb - vbe) / ( rb + re*(beta+1)   )

  ic = beta * ib

  vce = vcc - ( (rc*beta) + re*(beta+1)  )*ib

  hfe = beta

  hie = 0.026/ib

  gv = (hfe/hie) * ( (rc*rl)/(rc+rl) )

  gi = hfe*( rb / (rb + hie) )*(rc/(rc + rl))

  zi = ((rb*hie) / (rb + hie) )

  zo = rc

  fcb = 1/ (2*pi*rb*cb )
  fce = 1/ (2*pi*re*ce )
  fcc = 1/ (2*pi*rc*cc )

  f_list = [fcb, fce, fcc]
  f_min = max(f_list)

  vi_max = vcc/gv

  r_list = [vbb, vbe, ib, ic, vce, hfe, hie, gv, gi,
            zi, zo, fcb, fce, fcc, f_min , vi_max]

  return r_list

# ----------------------------------------------------
def amplificador_base_comun(vcc, beta, rb, re, rc, rl, cb, ce, cc):
  vbe = 0.7

  vbb = rb_vbb_calculator(r)
  vbb = vbb[1]

  ib = (vbb - vbe) / ( rb + re*(beta+1)   )

  ic = beta * ib

  vce = vcc - ( (rc*beta) + re*(beta+1)  )*ib

  hfb = (beta / (beta + 1))

  hib = 0.026/ ((beta+1)*ib)

  gv = (hfb/hib) * ( (rc*rl)/(rc+rl) )

  gi = hfb*( rc / (rc + rl) )*(re/(re+ hib))

  zi = ((re*hib) / (re + hib) )

  zo = rc

  fcb = 1/ (2*pi*rb*cb )
  fce = 1/ (2*pi*re*ce )
  fcc = 1/ (2*pi*rc*cc )

  f_list = [fcb, fce, fcc]
  f_min = max(f_list)

  vi_max = vcc/gv

  r_list = [vbb, vbe, ib, ic, vce, hfb, hib, gv, gi,
            zi, zo, fcb, fce, fcc, f_min , vi_max]

  return r_list
# ----------------------------------------------------


if config == "e":
  final_values = amplificador_emisor_comun(vcc, beta, rb, re, rc, rl, cb, ce, cc)
elif config == "b":
  final_values = amplificador_base_comun(vcc, beta, rb, re, rc, rl, cb, ce, cc)

else:
  print("No configuration selected")

final_list = value_initial + final_values

print("\nLos valores ingresados corresponden a: \n\n")
table.field_names = ["Quantity", "Measure"]

for i in range(len(names)):
  table.add_row([ "{0}".format(names[i])  ,  "{0:1.3E}".format(final_list[i]) ], divider = True)

print(table)


¿En qué configuración está el amplificador
 Emisor o base común? [e, b]: b
¿Cuál es la cantidad de resistencias en la divisora? [1,2,3]: 1

Los valores ingresados corresponden a: 


+----------+-----------+
| Quantity |  Measure  |
+----------+-----------+
|   VCC    | 9.000E+00 |
+----------+-----------+
|   BETA   | 2.000E+02 |
+----------+-----------+
|    R1    | 1.000E+06 |
+----------+-----------+
|    R2    | 8.200E+03 |
+----------+-----------+
|    R3    | 1.000E+04 |
+----------+-----------+
|    RB    | 1.000E+06 |
+----------+-----------+
|    RE    | 5.600E+02 |
+----------+-----------+
|    RC    | 3.900E+03 |
+----------+-----------+
|    RL    | 6.800E+03 |
+----------+-----------+
|    CB    | 2.000E-04 |
+----------+-----------+
|    CE    | 1.000E-05 |
+----------+-----------+
|    CC    | 1.000E-06 |
+----------+-----------+
|   VBB    | 9.000E+00 |
+----------+-----------+
|   VBE    | 7.000E-01 |
+----------+-----------+
|    IB    | 7.460E-06 |
+----------+------